In [1]:
import os

def base_name(title):
    """
    Removes the file extension from the image title safely.
    Example: 'cell1.tif' -> 'cell1'
    """
    return os.path.splitext(title)[0]

In [25]:
title = "MP_U2OS_fixed_20nMJF549_ORC1_MGS5.nd2 - MP_U2OS_fixed_20nMJF549_ORC1_MGS5.nd2 (series 1)"

In [ ]:
if "MP" in title and " - " in title:
    title = title.split("-")[1]
    title = title.replace(" ", "", 1) # delete fist blank
    title = title.replace(" ", "_") # repalce other blanks to underscore
    print("1", title)
else:
    title = base_name(title)
    print("2", title)

1  MP_U2OS_fixed_20nMJF549_ORC1_MGS5.nd2 (series 1)


In [ ]:
def _foci_image(imp, rois, p):
    '''
    This function process a single image
    imp - image
    p - parameters
    '''

	# Parameters

	# Processing image title
	img_title = imp.getTitle()
	base_title = img_title.split('.')[0] # remove extensionimg_title
    
	# Loop through the all ROIs of the image
	for i, roi in enumerate(rois):
		roi_name = roi.getName()
		if roi_name is None or roi_name.strip() == "":
			roi_name = "ROI_{}".format(i + 1)
		
		# Set ROI and crop image
		imp.setRoi(roi) # activate the current ROI
		dup = imp.crop() # crop image
		dup = dup.duplicate() # duplicate thi ROI to a new image
		dup.setTitle("ROI_{}".format(i + 1)) # set name to this ROI
		dup_name = dup.getTitle() # get duplicated image name name
		
		# Image processing
		#IJ.run(dup, "Subtract Background...", "rolling=4") # sybstact background 
	
		dup.show() # show the image 
	
		# Run ThunderSTORM plugin
		# threshold=3*std(Wave.F1)!
		try:
			IJ.run(dup,
			"Run analysis",
			"filter=[Wavelet filter (B-Spline)] scale=2.0 order=3 "
			"detector=[Local maximum] connectivity=8-neighbourhood threshold=3*std(Wave.F1) "
			"estimator=[PSF: Integrated Gaussian] sigma=1.6 fitradius=3 "
			"method=[Weighted Least squares] full_image_fitting=false mfaenabled=false "
			"renderer=[No Renderer]")
			
			# Export results in the .csv file
			#table_name = "{}_{}.csv".format(img_title[:-4], dup_name)
			#csv_path = os.path.join(output_dir, table_name)
			IJ.run(
				"Export results",
				"filepath={} fileformat=[CSV (comma separated)] "
				"sigma=false intensity=true chi2=false offset=false "
				"saveprotocol=false x=true y=true bkgstd=false "
				"id=true uncertainty=false frame=false".format(csv_path))
		    
			# Save cropped image
			file_name = "{}_{}.png".format(base_title, dup_name)
			#image_path = os.path.join(output_dir, file_name)
			#IJ.save(dup, image_path)
			
			# Close cropped image
			dup.close()
			
			# Close table with results
			windows = WindowManager.getNonImageWindows()
			for w in windows:
				if w.getTitle() == "ThunderSTORM: results":
					w.dispose()


		except Exception as e:
				IJ.log("Error running ThunderSTORM analysis for '{}': {}".format(roi_name, str(e)))


		return

In [30]:
import re

def make_key(filename):
    s = filename.lower()

    # remove leading channel prefix like "c1_" or "c2_"
    s = re.sub(r'^c\d+_', '', s)

    # remove optional "(series_XX)" part if present
    #s = re.sub(r'\(series_\d+\)', '', s)

    # remove roi suffix
    s = re.sub(r'_rois(?=\.|$)', '', s)

    # strip known extensions repeatedly (handles ".nd2.jpg", ".ome.tif", etc.)
    while True:
        new = re.sub(r'(\.ome)?\.(tif|tiff|png|jpg|jpeg|zip|nd2|czi|lif)$', '', s)
        if new == s:
            break
        s = new

    # cleanup leftover underscores/spaces
    s = re.sub(r'[\s_]+$', '', s)
    s = re.sub(r'^[\s_]+', '', s)

    return s

In [ ]:
images = [
    u'C2_MP_U2OS_fixed_20nMJF549_ORC1_MGS5.nd2_(series_01).jpg',
    u'C2_MP_U2OS_fixed_20nMJF549_ORC1_MGS5.nd2_(series_02).jpg'
]

rois = [
    u'C1_MP_U2OS_fixed_20nMJF549_ORC1_MGS5.nd2_(series_01)_rois.zip',
    u'C1_MP_U2OS_fixed_20nMJF549_ORC1_MGS5.nd2_(series_02)_rois.zip'
]

roi_map = {}
for r in rois:
    k = make_key(r)
    roi_map.setdefault(k, []).append(r)

# ---- make pairs ----
pairs = []
unmatched_images = []

for img in images:
    k = make_key(img)
    if k in roi_map and len(roi_map[k]) > 0:
        roi_file = roi_map[k].pop(0)  # take one matching roi
        pairs.append((img, roi_file))
    else:
        unmatched_images.append(img)

mp_u2os_fixed_20nmjf549_orc1_mgs5.nd2_(series_01)
mp_u2os_fixed_20nmjf549_orc1_mgs5.nd2_(series_02)


In [32]:
pairs

[('C2_MP_U2OS_fixed_20nMJF549_ORC1_MGS5.nd2_(series_01).jpg',
  'C1_MP_U2OS_fixed_20nMJF549_ORC1_MGS5.nd2_(series_01)_rois.zip'),
 ('C2_MP_U2OS_fixed_20nMJF549_ORC1_MGS5.nd2_(series_02).jpg',
  'C1_MP_U2OS_fixed_20nMJF549_ORC1_MGS5.nd2_(series_02)_rois.zip')]

In [28]:
roi_map

{'c1_mp_u2os_fixed_20nmjf549_orc1_mgs5.nd2_(series_01)': ['C1_MP_U2OS_fixed_20nMJF549_ORC1_MGS5.nd2_(series_01)_rois.zip'],
 'c1_mp_u2os_fixed_20nmjf549_orc1_mgs5.nd2_(series_02)': ['C1_MP_U2OS_fixed_20nMJF549_ORC1_MGS5.nd2_(series_02)_rois.zip']}